In [9]:
from mpi4py import MPI
import json

# read parameters
params =

comm = MPI.COMM_WORLD
mode = MPI.MODE_RDONLY
fh = MPI.File.Open(comm, "data-big.txt", mode)

# set read size parameter
file_size = fh.Get_size()
num_process = comm.Get_size()
rank = comm.Get_rank()
chunk_size = int(file_size / num_process)
start = rank * chunk_size + 1

# read all
buf = bytearray(chunk_size)
fh.Iread_at(start, buf)

raw = buf.decode()

print("Process", rank, "finish reading. Total size is", len(raw), "bytes.")

Exception: Invalid argument, error stack:
MPI_FILE_IREAD_AT(100): Invalid offset argument

In [22]:
import json
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
fh = MPI.File.Open(comm, "config", MPI.MODE_RDONLY)
file_size = fh.Get_size()
buf = bytearray(file_size)
fh.Iread_at(0, buf)
params = json.loads(buf.decode())

# with open("config", "r") as f:
#     params = json.load(f)

print("Process", rank, "finish", "Answer: ", params)

Process 0 finish Answer:  {'chunk_size': 100000}


In [2]:
ts = [(x[0], float(x[1]), int(x[2])) for row in raw.split("\n") for x in [row.split(",")] if len(x) == 3]
print(len(ts))
print(ts[:5])
print(ts[-5:])

KeyboardInterrupt: 

In [4]:
raw[:100]

'20140804:10:00:00.574914,1173.56,471577\n20140804:10:00:00.898688,1251.60,445361\n20140804:10:00:00.94'

In [133]:
# find out the longest misplacement

tmp = [x[0] for x in ts]
rank = np.argsort(tmp)
rev = np.maximum(rank - np.arange(len(tmp)), 0)
cdf = np.sort(rev)

print(cdf[99000]) # 99% quantile is 871
print(cdf[99900]) # 99.9% quantile is 1085
print(cdf[-1]) # 100% quantile is 99882
print(np.sum(cdf>0)/len(cdf)) # 50% of records are misplaced

871
1085
99882
0.49394


In [136]:
# duplicates - record, time, (price, vol)
# a paper about filtering tick data
# erratic price on thin volume
# t2 from (t1, t2, t3) is out of wreck. use brownian bridge?